In [ ]:
import numpy as np
import pandas as pd

In [ ]:
sentiment = pd.read_csv("data/tweets_sentiment.csv")
sentiment.drop(sentiment.columns[[0]], axis=1, inplace=True)
sentiment.rename(columns = {'created_at':'Date'}, inplace = True)
spy = pd.read_csv("data/SPY.csv")

In [ ]:
df = pd.merge(sentiment, spy, on ='Date', how ="inner")

In [ ]:
opn = {'Date': [df['Date'][i] for i in range(3,len(df))], \
       'Movement': [1 if df['Open'][i] - df['Open'][i-1] > 0 else 0 for i in range(3,len(df))], \
       'Open1': [df['Open'][i-1] for i in range(3,len(df))], \
       'Open2': [df['Open'][i-2] for i in range(3,len(df))], \
       'Open3': [df['Open'][i-3] for i in range(3,len(df))]}

sen = {'Date': [df['Date'][i] for i in range(3,len(df))], \
       'Sentiment1': [df['compound'][i-1] for i in range(3,len(df))], \
       'Sentiment2': [df['compound'][i-2] for i in range(3,len(df))], \
       'Sentiment3': [df['compound'][i-3] for i in range(3,len(df))]}

In [ ]:
o = pd.DataFrame(opn)
s = pd.DataFrame(sen)
data = pd.merge(o, s, on='Date', how='inner')

In [ ]:
data.head()

## Training Model

In [ ]:
from sklearn.model_selection import train_test_split
X = data[data.columns[2:]]
y = data[data.columns[1]]
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.3)

In [ ]:
positives = sum([1 if y[i] == 1 else 0 for i in range(len(y))])
negatives = len(y) - positives
print("Positive: ", positives)
print("Negative: ", negatives)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

## Testing different kernels

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
kernels = ['linear', 'rbf', 'poly', 'sigmoid']
acc = {}
pred = {}

In [ ]:
for k in kernels:
    classifier = SVC(kernel=k)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    acc[k] = accuracy_score(y_test, y_pred)
    pred[k] = y_pred
    

In [ ]:
from sklearn.metrics import classification_report
best_kernels = [k for k,v in acc.items() if v == max(acc.values())]
print(best_kernels)
print("Classification report of", best_kernels[0], "kernel")
print(classification_report(y_test, pred[best_kernels[0]]));

## Comparing Best Model with SVM Trained Only on Price

In [ ]:
X_train = [p[:3] for p in X_train]
X_test = [p[:3] for p in X_test]

In [ ]:
classifier = SVC(kernel=best_kernels[0])
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print("Classification report of", best_kernels[0], "kernel")
print(classification_report(y_test,y_pred));